# Chapin style converstaion generator

In [ ]:
import pandas as pd

## Transforming data into a script like format

In [ ]:
data_path = "data/velmaxdata.csv"
threads = pd.read_csv(data_path)

In [ ]:
threads.head()

In [ ]:
fields_to_drop = ['Unnamed: 0','date']
threads = threads.drop(fields_to_drop,axis=1)
threads.head()

### Building script blocks

In [ ]:
thread_id = threads[0:1]['threadid'][0]
print(thread_id)

In [ ]:
with open('./data/script.txt','w') as out_file:
    
    for index, thread in threads.iterrows():
        
        if thread['threadid'] != thread_id:
            out_file.write('\n\n')

        line = "{}: {}".format(thread['author'],thread['text']).strip()
        if not line.endswith('.'):
            line+='.'

        out_file.write(line+'\n')
        
        thread_id = thread['threadid']

## Checkpoint 

After processing the data you can start here loading the transforming data directly. The preprocessed data has been saved to disk.

In [1]:
import helper
data_dir = './data/script.txt'

In [2]:
text = helper.load_data(data_dir)
index = len(text)//500
text = text[:index]

## Explore the data

In [3]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 35915
Number of scenes: 1554
Average number of sentences in each scene: 10.297297297297296
Number of lines: 17556
Average number of words in each line: 15.940248348143085

The sentences 0 to 10:
Mitsu-Elias: Cuando lo acelero hace un ruidito que juro ke no identifico que es, suena como si las bujias estubieran humedas, cosa que ya me asegure que no fuera. Que podria ser? Otra cosa, que beneficios me puede dar comprar un catalizador de marca y cuanto vale mas o menos.
Thunderboy: Pues si vos no sabes nosotros menos!! akfdjadsfkljkalsfd.
Mitsu-Elias: Pues si vos no sabes nosotros menos!! akfdjadsfkljkalsfd.
Thunderboy: Pues si, mas de una vez se me han bajado las revoluciones y ha perdido potencia, cuando lo compre venia con una bujia diferente a las demas y por descuido no me fije en eso, pero cambie las bujias y ahora son iguales, no se si tendra que ver que se me hayan mojado mas de una vez (con esos diluvios que caen por aki), kisa ti

## Implementing Preprocessing Functions

In order to preprocess the dataset we are going to implement the following preprocessing functions:
* Lookup Table
* Tokenize Punctuation

### Lookup Table

To create a word embedding, we need to transform the words to ids. We will create two dictionaries:
* Dictionary to go from the words to an id, we'll call vocab_to_int
* Dictionary to go from the id to word, we'll call int_to_vocab

In [4]:
import numpy as np
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of velmax scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    print('countint words')
    word_counts = Counter(text)
    print('sorting words')
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    print('generating int to vocab')
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    print('generating vocab to int')
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

## Tokenize punctuation

We'll be splitting the script into a word array using spaces as delimiters. However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

The token_lookup function will return a dict that will be used to tokenize symbols like "?" into "||Question_Mark||"

This dictionary will be used to token the symbols and add the delimiter (space) around it. This separates the symbols as it's own word, making it easier for the neural network to predict on the next word

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_dic = {
        '.' : '||Period||',
        ',' : '||Comma||',
        '"' : '||Quotation_Mark||',
        ';' : '||Semicolon||',
        '!' : '||Exclamation_Mark||',
        '?' : '||Question_Mark||',
        '(' : '||Left_Parentheses||',
        ')' : '||Right_Parentheses||',
        '--': '||Dash||',
        '\n': '||Return||'
    }
    return punct_dic

# Pre process all data and save it

In [6]:
import helper
data_dir = './data/script.txt'
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

loading data set
reducing (3)
creating token punctuation dict
replacing tokens
to lower case
spliting text
creating lookup tables
countint words
sorting words
generating int to vocab
generating vocab to int
saving data.
writing total_bytes=1325693...
writing bytes [0, 1325693)... done.


## Checkpoint #2

The preprocessed data has been saved to disk. We can start from here the next time

In [1]:
import helper
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Check Tensorflow version

In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Get tensors

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs_ = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return inputs_, targets, learning_rate, keep_prob

## Build RNN Cell and Initialize

In [4]:
def make_cell(rnn_size, keep_prob):
    lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

def get_init_cell(batch_size, rnn_size, keep_prob):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    num_layers = 1
    cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return cell, initial_state

## Word embedding

Apply embedding to input_data using TensorFlow. Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

## Build RNN

In [6]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state

## Build the Neural Network

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    #Apply embedding to input
    embeddings = get_embed(input_data, vocab_size, embed_dim)
    
    #Build RNN
    output, final_state = build_rnn(cell, embeddings)
    
    
    # add fully connected layer 
    # Setting activation function as None will implement a linear activation function
    
    #weights and biases
    weights = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
    biases = tf.zeros_initializer()
    logits = tf.contrib.layers.fully_connected(output, 
                                               vocab_size, 
                                               activation_fn=None,
                                               weights_initializer=weights,
                                               biases_initializer=biases)
    
    return logits, final_state

## Batches

For exmple, get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) would return a Numpy array of the following:
```
[
  #  First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, 1. This is a common technique used when creating sequence batches, although it is rather unintuitive.


In [8]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))
    inputs = np.array(int_text[: n_batches * batch_size * seq_length])
    outputs = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x = np.split(inputs.reshape(batch_size, -1), n_batches, 1)
    y = np.split(outputs.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x, y)))

## Neural Network Training

### Hyperparameters

Tune the following parameters:
- Set num_epochs to the number of epochs.
- Set batch_size to the batch size.
- Set rnn_size to the size of the RNNs.
- Set embed_dim to the size of the embedding.
- Set seq_length to the length of sequence.
- Set learning_rate to the learning rate.
- Set show_every_n_batches to the number of batches the neural network should print progress.

In [10]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 512
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.006
# Show stats for every n number of batches
show_every_n_batches = 10
#Keep probability
keep_probability = 1

save_dir = './save'

## Build the graph

In [11]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr, keep_prob = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, keep_prob)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Training

In [12]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate,
                keep_prob: keep_probability
            }
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/95   train_loss = 10.037
Epoch   0 Batch   10/95   train_loss = 6.411
Epoch   0 Batch   20/95   train_loss = 5.966
Epoch   0 Batch   30/95   train_loss = 5.665
Epoch   0 Batch   40/95   train_loss = 5.534
Epoch   0 Batch   50/95   train_loss = 5.328
Epoch   0 Batch   60/95   train_loss = 5.256
Epoch   0 Batch   70/95   train_loss = 5.213
Epoch   0 Batch   80/95   train_loss = 5.139
Epoch   0 Batch   90/95   train_loss = 4.952
Epoch   1 Batch    5/95   train_loss = 4.738
Epoch   1 Batch   15/95   train_loss = 4.587
Epoch   1 Batch   25/95   train_loss = 4.399
Epoch   1 Batch   35/95   train_loss = 4.410
Epoch   1 Batch   45/95   train_loss = 4.159
Epoch   1 Batch   55/95   train_loss = 4.174
Epoch   1 Batch   65/95   train_loss = 4.040
Epoch   1 Batch   75/95   train_loss = 4.087
Epoch   1 Batch   85/95   train_loss = 4.104
Epoch   2 Batch    0/95   train_loss = 3.783
Epoch   2 Batch   10/95   train_loss = 3.759
Epoch   2 Batch   20/95   train_loss = 3.749
Epoch   2

## Save Parameters

Save seq_length and save_dir for generating a new Velmax Thread

In [13]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

## Checkpoint

In [14]:
import tensorflow as tf
import numpy as np
import helper

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions

### Get Tensors

Get tensors from loaded_graph using the function get_tensor_by_name(). Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

In [15]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    inputs = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probs = loaded_graph.get_tensor_by_name("probs:0")
    keep_prob = loaded_graph.get_tensor_by_name("keep_prob:0")
    return inputs, initial_state, final_state, probs, keep_prob

## Choose Word

Implement the pick_word() function to select the next word using probabilities

In [16]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    probabilities = np.squeeze(probabilities)
    probabilities[np.argsort(probabilities)[:-5]] = 0
    probabilities = probabilities / np.sum(probabilities)
    return np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]

## Generate Velmax Thread

This will generate the Velmax Thread. We will set gen_length to the length of Thread we want to generate.

In [17]:
gen_length = 300
# username from threads
prime_word = 'hola'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs, keep_prob = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state, keep_prob: 1})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    velmax_thread = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        velmax_thread = velmax_thread.replace(' ' + token.lower(), key)
    velmax_thread = velmax_thread.replace('\n ', '\n')
    velmax_thread = velmax_thread.replace('( ', '(')
        
    print(velmax_thread)

INFO:tensorflow:Restoring parameters from ./save
hola mira te mande correo con la papeleria de mi novia.. gracias.
princesaranita: si yo en la mañana a dejar hoy si me quedo asi como de tu casa, de todos modos vas a trabajar a la cantera.. porque hay que hacer algo extra :si:.
afigueredo88: gracias, buena onda.
pablo mp3: mucha!.
danielmoc: te envie mi cv. oralex.
anita la chinita: sin limites... está a la plaza...
agphone: pm chavos.
brenic: y de que es lo que te van a enviar y no es mi,. : vos lo que pasa es que tengo un cacho de experiencia en ventas, por si te interesa es de algo me avisas.
charles gyborin: mucha si eso es cierto. cuanto se gana el trabaja o call center que dias, si hay rango de edades esta hasta el 2.
shadow_ap: que onda me mandan para la info!!!.
makillo: uta!! aki en q. 300.
cheves: yo se de alguien, cuando puede llegar? hoy?.
huguinho: hasta el doble de eso?.
pabloyoc: los sueldos del foro que asi que me mando su cv le urge gracias.
alexlm: grax men.
jruizw: a 